In [224]:
import pandas as pd
import numpy as np

In [268]:
all_tickets = pd.read_csv('all_tickets.csv')
chair_mapping = pd.read_excel('chair_mapping_20240801.xlsx')
sf = pd.read_csv('extract.csv')    

installments = pd.read_csv('HapoelInstallmentsReportCsv_2025-08-13.csv')

installments = installments.rename(columns={'InstallmentTransactionId':'Transaction',
                                            'Installment Ticket Id': 'Return ID'})

/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_34322/2960671920.py:1: DtypeWarning: Columns (0,12,32,48) have mixed types. Specify dtype option on import or set low_memory=False.
  all_tickets = pd.read_csv('all_tickets.csv')


## Separate data by type:

In [269]:
print('all_tickets shape:', all_tickets.shape)

all_tickets shape: (74425, 52)


In [270]:
all_tickets = all_tickets.loc[~all_tickets['Product'].str.contains('|'.join(['חניה', 'חניית', 'חיצונית']), na=False)]

return_dat = all_tickets[(all_tickets['Type'] == 'Return') & (all_tickets['Status'] == 'Canceled')]
print('return_dat shape:', return_dat.shape)

sale_dat = all_tickets[(all_tickets['Type'] == 'Sale') & (all_tickets['Status'] == 'Canceled')]
print('sale_dat shape:', sale_dat.shape)


return_dat shape: (4107, 52)
sale_dat shape: (4372, 52)


### Filtering games:

In [271]:
all_products = all_tickets.loc[~all_tickets['Product'].str.contains('|'.join(['חניה', 'חניית', 'חיצונית']), na=False), 'Product'].unique()

In [272]:
print('Installments shape before:', installments.shape)
installments = installments[installments['InstallmentProducts'].isin(all_products)]
print('Installments shape after:', installments.shape)

Installments shape before: (161593, 10)
Installments shape after: (77002, 10)


### separate to different use cases:

In [273]:
df = return_dat.copy()

# add count column that counts the occurrences of each product in transaction
df['count'] = (
    df.groupby(['Transaction', 'Product'])['Transaction']
      .transform('size')
)

multi = df[df['count'] > 1]
print('Rows with more than one product:', multi.shape)

single = df[df['count'] == 1]
print('Rows with exactly one product:', single.shape)

Rows with more than one product: (3492, 53)
Rows with exactly one product: (615, 53)


## Merge single table with return_dat:

In [274]:
single['Transaction'] = single['Transaction'].astype(int)
installments['Transaction'] = installments['Transaction'].astype(int)
return_single_table = single.merge(installments[['Return ID', 'Transaction']], on='Transaction', how='left')

return_single_table

/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_34322/1431468344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single['Transaction'] = single['Transaction'].astype(int)


,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Delivery type,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher value,Unnamed: 51,count,Return ID
0,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3041925.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,3041943.0
1,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3041925.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,183324.0
2,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3037182.0,ברק קפלן,41186.0,322395716,ברק,קפלן,barakkaplan@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,183323.0
3,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3040621.0,ברק קפלן,41186.0,322395716,ברק,קפלן,barakkaplan@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,183322.0
4,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3040686.0,הפועל ירושלים,41757.0,43467752,הפועל,ירושלים,hj@hapoel.co.il,...,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,183296.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,4172,גמר ליגת ווינר משחק 2: מכבי תל אביב 🏠,Canceled,3391181.0,עינת מידן-דוד,26452.0,43552660,עינת,מידן-דוד,einatth@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,233912.0
623,4237,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,Canceled,3439035.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,236860.0
624,4237,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,Canceled,3439036.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,236859.0
625,4237,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,Canceled,3439038.0,עדו מידן דוד,16541.0,43467760,עדו,מידן דוד,idomedan@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,236858.0


## Modify because shape isnt the same:

In [275]:
return_single_table['num_char'] = return_single_table['Return ID'].astype(str).str.len()
return_single_table = return_single_table[return_single_table['num_char'] == 8].drop(columns=['num_char'])

return_single_table = return_single_table[return_single_table['Return ID'] != 197315.0]
return_single_table

,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Delivery type,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher value,Unnamed: 51,count,Return ID
1,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3041925.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,183324.0
2,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3037182.0,ברק קפלן,41186.0,322395716,ברק,קפלן,barakkaplan@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,183323.0
3,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3040621.0,ברק קפלן,41186.0,322395716,ברק,קפלן,barakkaplan@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,183322.0
4,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3040686.0,הפועל ירושלים,41757.0,43467752,הפועל,ירושלים,hj@hapoel.co.il,...,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,183296.0
5,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3040908.0,הפועל ירושלים,41757.0,43467752,הפועל,ירושלים,hj@hapoel.co.il,...,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,183295.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,4172,גמר ליגת ווינר משחק 2: מכבי תל אביב 🏠,Canceled,3391181.0,עינת מידן-דוד,26452.0,43552660,עינת,מידן-דוד,einatth@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,233912.0
623,4237,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,Canceled,3439035.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,236860.0
624,4237,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,Canceled,3439036.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,236859.0
625,4237,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,Canceled,3439038.0,עדו מידן דוד,16541.0,43467760,עדו,מידן דוד,idomedan@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,236858.0


### Summing the number of tickets and comparing it to the number of unique prices:

In [276]:
print('return_dat.shape:', return_dat.shape)
print('multi.shape:', multi.shape)

return_dat.shape: (4107, 52)
multi.shape: (3492, 53)


### Sum the number of tickets per transaction:

In [277]:
# return_dat["TransactionCount"] = return_dat["Transaction"].map(return_dat["Transaction"].value_counts())
# return_dat

In [278]:
unique_prices = multi.groupby("Transaction")["Price"].nunique().sort_values(ascending=False).reset_index()
unique_prices.rename(columns={"Price": "Unique Prices Count"}, inplace=True)
unique_prices

,Transaction,Unique Prices Count
0,3695556.0,4
1,3740859.0,3
2,3699807.0,3
3,3743553.0,2
4,3605993.0,2
...,...,...
347,3606026.0,1
348,3606025.0,1
349,3606021.0,1
350,3605802.0,1


In [279]:
transaction_count = multi.groupby('Transaction')['Id'].count().reset_index()
transaction_count.rename(columns={'Id': 'Tickets in transaction'}, inplace=True)
transaction_count

,Transaction,Tickets in transaction
0,3510611.0,2
1,3510842.0,2
2,3512275.0,2
3,3513049.0,2
4,3513762.0,5
...,...,...
347,3744457.0,2
348,3744462.0,2
349,3744543.0,2
350,3744586.0,2


In [280]:
merged = transaction_count.merge(unique_prices, on="Transaction", how="inner")

merged['equal'] = merged['Tickets in transaction'] == merged['Unique Prices Count']

merged

,Transaction,Tickets in transaction,Unique Prices Count,equal
0,3510611.0,2,1,False
1,3510842.0,2,1,False
2,3512275.0,2,1,False
3,3513049.0,2,1,False
4,3513762.0,5,1,False
...,...,...,...,...
347,3744457.0,2,1,False
348,3744462.0,2,1,False
349,3744543.0,2,1,False
350,3744586.0,2,1,False


In [281]:
merged.equal.value_counts()

equal
False    344
True       8
Name: count, dtype: int64

In [282]:
multi_equal_price = merged[merged['equal']]

multi_price_not_equal = merged[merged['equal'] == False]

multi_with_equal_price = multi[multi['Transaction'].isin(multi_equal_price['Transaction'])]

In [283]:
multi_with_equal_price = multi_with_equal_price.merge(installments[['Return ID', 'Transaction', 'InstallmentProductPrice']],
                                                         left_on=["Transaction", 'Price'], right_on=["Transaction", 'InstallmentProductPrice'], how="left")


multi_with_equal_price                                                         

,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher value,Unnamed: 51,count,Return ID,InstallmentProductPrice
0,1694,Round 2: Hapoel Tel Aviv 🚗,Canceled,3059311.0,יהונתן זהר,1357401.0,220342414,יהונתן,זהר,yehonatan.zohar.7@gmail.com,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,188240.0,-70.0
1,1694,Round 2: Hapoel Tel Aviv 🚗,Canceled,3059312.0,יאיר זהר,1357402.0,025609959,יאיר,זהר,NaN,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,188241.0,-100.0
2,2684,"🏠 מחזור11: הפועל ״בנק יהב״ ירושלים -מכבי תא""",Canceled,3133492.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,209949.0,-70.0
3,2684,"🏠 מחזור11: הפועל ״בנק יהב״ ירושלים -מכבי תא""",Canceled,3133491.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,209950.0,-100.0
4,3014,"מחזור 14: הפועל תא 🚗""",Canceled,3169872.0,גיא עינב,19270.0,32077943,גיא,עינב,guy.einav@mfa.gov.il,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,214276.0,-80.0
5,3014,"מחזור 14: הפועל תא 🚗""",Canceled,3169873.0,מתן עינב,1331280.0,329707202,מתן,עינב,mataneinav08@gmail.com,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,214277.0,-50.0
6,3476,ליגת ווינר סל מחזור 20: הפועל חיפה 🏠,Canceled,3279691.0,אריאל חוה,1418333.0,341757375,אריאל,חוה,NaN,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,227362.0,-60.0
7,3476,ליגת ווינר סל מחזור 20: הפועל חיפה 🏠,Canceled,3279690.0,איתיאל חוה,1410624.0,036260321,איתיאל,חוה,etlh79@gmail.com,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,227361.0,-90.0
8,3575,מחזור 22: אליצור עירוני נתניה 🏠,Canceled,3286267.0,מתניה גבלר,1419324.0,342045937,מתניה,גבלר,NaN,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,227880.0,-80.0
9,3575,מחזור 22: אליצור עירוני נתניה 🏠,Canceled,3286266.0,בנימין גבלר,1365237.0,326821022,בנימין,גבלר,bengabler@gmail.com,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,227881.0,-110.0


## Merge multi_with_equal_price with return_single_table:

In [284]:
print('multi_with_equal_price shape:', multi_with_equal_price.shape)
print('return_single_table shape:', return_single_table.shape)

multi_with_equal_price shape: (16, 55)
return_single_table shape: (615, 54)


In [285]:
all = pd.concat([multi_with_equal_price, return_single_table], ignore_index=True)
all

,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher value,Unnamed: 51,count,Return ID,InstallmentProductPrice
0,1694,Round 2: Hapoel Tel Aviv 🚗,Canceled,3059311.0,יהונתן זהר,1357401.0,220342414,יהונתן,זהר,yehonatan.zohar.7@gmail.com,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,188240.0,-70.0
1,1694,Round 2: Hapoel Tel Aviv 🚗,Canceled,3059312.0,יאיר זהר,1357402.0,025609959,יאיר,זהר,NaN,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,188241.0,-100.0
2,2684,"🏠 מחזור11: הפועל ״בנק יהב״ ירושלים -מכבי תא""",Canceled,3133492.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,209949.0,-70.0
3,2684,"🏠 מחזור11: הפועל ״בנק יהב״ ירושלים -מכבי תא""",Canceled,3133491.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,209950.0,-100.0
4,3014,"מחזור 14: הפועל תא 🚗""",Canceled,3169872.0,גיא עינב,19270.0,32077943,גיא,עינב,guy.einav@mfa.gov.il,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,214276.0,-80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,4172,גמר ליגת ווינר משחק 2: מכבי תל אביב 🏠,Canceled,3391181.0,עינת מידן-דוד,26452.0,43552660,עינת,מידן-דוד,einatth@gmail.com,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,233912.0,NaN
627,4237,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,Canceled,3439035.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,236860.0,NaN
628,4237,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,Canceled,3439036.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,236859.0,NaN
629,4237,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,Canceled,3439038.0,עדו מידן דוד,16541.0,43467760,עדו,מידן דוד,idomedan@gmail.com,...,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,236858.0,NaN


In [219]:
multi_equal_not_price.merge(installments[['Return ID', 'Transaction']],
                                                         left_on=["Transaction"], right_on=["Transaction"], how="left")

,Transaction,Tickets in transaction,Unique Prices Count,equal,Return ID
0,3510611.0,2,1,False,3035559.0
1,3510611.0,2,1,False,3035560.0
2,3510611.0,2,1,False,3035561.0
3,3510611.0,2,1,False,3035558.0
4,3510611.0,2,1,False,181739.0
...,...,...,...,...,...
3581,3744795.0,60,1,False,236801.0
3582,3744795.0,60,1,False,236800.0
3583,3744795.0,60,1,False,236799.0
3584,3744795.0,60,1,False,236798.0


In [ ]:
multi_equal_not_price

,Transaction,Tickets in transaction,Unique Prices Count,equal
0,3510611.0,2,1,False
1,3510842.0,2,1,False
2,3512275.0,2,1,False
3,3513049.0,2,1,False
4,3513762.0,5,1,False
...,...,...,...,...
347,3744457.0,2,1,False
348,3744462.0,2,1,False
349,3744543.0,2,1,False
350,3744586.0,2,1,False


In [27]:
cols = ['Transaction', 'Id', 'Type', 'Id', 'Status', 'Price']

In [29]:
dat = pd.concat([all_tickets[all_tickets['Transaction'] == 3514971], all_tickets[all_tickets['Transaction'] == 3513200]]).sort_values('Id').reset_index(drop=True)
dat

,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Payment type,Payment method,Delivery type,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher value,Unnamed: 51
0,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN
1,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN
2,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN
3,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN
4,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039244.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN
5,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039244.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN
6,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039245.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN
7,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039245.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN


In [47]:
return_dat = dat[(dat['Type'] == 'Return') & (dat['Status'] == 'Canceled')]
sale_dat = dat[(dat['Type'] == 'Sale') & (dat['Status'] == 'Canceled')]

In [48]:
return_dat

,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Payment type,Payment method,Delivery type,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher value,Unnamed: 51
0,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN
2,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN
4,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039244.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN
6,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039245.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN


In [49]:
return_dat['running_number'] = return_dat.groupby('Transaction').cumcount() + 1

/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_26168/1218201186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return_dat['running_number'] = return_dat.groupby('Transaction').cumcount() + 1


In [50]:
return_dat

,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Payment method,Delivery type,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher value,Unnamed: 51,running_number
0,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1
2,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2
4,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039244.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,3
6,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039245.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,4


In [52]:
return_dat.merge(installments[['Transaction', 'Return ID', 'running_number']], on=['Transaction', 'running_number'], how='left')

,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Delivery type,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher value,Unnamed: 51,running_number,Return ID
0,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,1,NaN
1,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,2,183358.0
2,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039244.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,3,183355.0
3,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039245.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash desk ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,4,183356.0


In [32]:
sale_dat

,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Payment type,Payment method,Delivery type,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher value,Unnamed: 51
1,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN
3,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN
5,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039244.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN
7,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039245.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN


In [51]:
installments = pd.read_csv('HapoelInstallmentsReportCsv_2025-08-13.csv')

installments = installments.rename(columns={'InstallmentTransactionId':'Transaction',
                                            'Installment Ticket Id': 'Return ID'})

installments['running_number'] = installments.groupby('Transaction').cumcount() + 1
installments

,Return ID,Transaction,InstallmentDate,InstallmentValueDate,InstallmentProducts,InstallmentPaymentPrice,InstallmentProductPrice,InstallmentPaymentExtRef,InstallmentProductExtRef,Installments,running_number
0,NaN,3596914,01/01/2025,01/01/2025,Other Payment,0.0,NaN,Other,NaN,NaN,1
1,3163501.0,3596914,01/01/2025,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN,2
2,3163500.0,3596914,01/01/2025,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN,3
3,NaN,3596915,01/01/2025,01/01/2025,Other Payment,0.0,NaN,Other,NaN,NaN,1
4,3163497.0,3596915,01/01/2025,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...
161588,3163489.0,3596912,31/12/2024,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN,4
161589,3163490.0,3596912,31/12/2024,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN,5
161590,3163491.0,3596912,31/12/2024,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN,6
161591,NaN,3596913,31/12/2024,31/12/2024,Other Payment,0.0,NaN,Other,NaN,NaN,1


In [36]:
sale_dat.merge(installments[['Transaction', 'Return ID']], on='Transaction', how='left')

,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Payment method,Delivery type,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher value,Unnamed: 51,Return ID
0,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN,NaN
1,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN,3039242.0
2,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN,3039243.0
3,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN,3039244.0
4,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039242.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN,3039245.0
5,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN,NaN
6,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN,3039242.0
7,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN,3039243.0
8,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN,3039244.0
9,1331,שלב הבתים משחק 1: עירוני נס ציונה,Canceled,3039243.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Cash,Cash desk ticket,NaN,בדיקה - מכירת כרטיסים מזומן ללא פרסונליזציה,Administrator,NaN,NaN,NaN,NaN,3039245.0


In [3]:
all_tickets = all_tickets[all_tickets['Type'] == 'Return']

all_tickets = all_tickets.rename(columns={
    'Product Id': 'Product ID',
    'Product': 'Asset Name',
    'Id': 'Roboticket ID',
    'Price area': 'Price Area Name',
    'Ticket price types': 'Price Name',
    'Base price': 'Base Price',
    'Area': 'Sector',
    'Type': 'Operation Name',
    'Date.1': 'Product Date',
    'User Id': 'Owner User ID',
    'assign using  ID number': 'Owner Identifier'
})

# Add missing columns
new_columns = [
    'Season ID', 'From', 'To', 'Stadium ID', 'Product Type', 'Product Name', 'Operation', 'Return Type', 'Return ID',
    'Platform', 'Entrance Text', 'Seat ID', 'Price Modifier Value', 'Access Token', 'Ticket ID', 'Transaction Owner',
    'Description', 'Account', 'Account ID', 'Contact', 'Product', 'Purchaser Account',
    'Return Asset', 'Status', 'Is Ticket', 'Is Season Ticket', 'Created By', 'Last Modified By'
]
for col in new_columns:
    if col not in all_tickets.columns:
        all_tickets[col] = None

# Reorder columns
desired_columns = [
    'Product ID' ,'Season ID', 'Asset Name', 'Roboticket ID', 'Price', 'Price Name', 'Price Area Name', 'Base Price', 'From', 'To',
    'Stadium ID', 'Sector', 'Row', 'Number', 'Product Type', 'Operation Name', 'Product Name', 'Product Date',
    'Operation', 'Return Type', 'Return ID', 'Owner User ID', 'Owner Identifier', 'Platform', 'Entrance Text',
    'Seat ID', 'Price Modifier Value', 'Access Token', 'Ticket ID', 'Transaction Owner', 'Purchaser Account', 'Stadium',
    'Description', 'Account', 'Account ID', 'Contact', 'Product', 'Transaction', 'Return Asset', 'Status', 'Is Ticket',
    'Is Season Ticket', 'Created By', 'Last Modified By'
]
for col in desired_columns:
    if col not in all_tickets.columns:
        all_tickets[col] = None
all_tickets = all_tickets[desired_columns]

# Set default values
all_tickets['Season ID'] = 24
all_tickets['Product Name'] = all_tickets['Asset Name']
all_tickets['Product Type'] = 'Match'
all_tickets['Purchaser Account'] = all_tickets['Owner User ID']
all_tickets['Is Ticket'] = True
all_tickets['Is Season Ticket'] = False
all_tickets['Stadium ID'] = np.where(
    all_tickets['Stadium'] == 'פיס ארנה', 1, all_tickets['Stadium ID']
)


# Define mapping function for Hebrew sectors
def map_sector_to_section_name(sector):
    sector = str(sector).strip()
    if sector.startswith('אולם'):
        number = ''.join(filter(str.isdigit, sector))
        return f'Court_{number}' if number else None
    elif 'תא צפוני' in sector:
        return 'Suite_21'
    elif 'תא צפיה' in sector:
        number = ''.join(filter(str.isdigit, sector))
        if number and 1 <= int(number) <= 16:
            return f'Suite_{number}'
    elif 'פרקט דרומי' in sector:
        return 'Court_side_17'
    elif 'פרקט צפוני' in sector:
        return 'Court_side_19'
    elif 'פרקט מרכזי' in sector or 'פרקט מרכז' in sector:
        return 'Court_side_18'
    elif 'גלריה' in sector:
        number = ''.join(filter(str.isdigit, sector))
        return f'Gallery_{number}' if number else None
    elif 'Courtyard' in sector:
        return 'Unmarked_12'
    return None

# Map section name
all_tickets['section_name'] = all_tickets['Sector'].apply(map_sector_to_section_name)
all_tickets['Row'] = all_tickets['Row'].astype(str).str.strip()
all_tickets['Number'] = all_tickets['Number'].astype(str).str.strip()

# Load chair mapping
chair_mapping['row_number'] = chair_mapping['row_number'].astype(str).str.strip()
chair_mapping['seat_number'] = chair_mapping['seat_number'].astype(str).str.strip()

# Merge to get Seat ID
all_tickets = all_tickets.merge(
    chair_mapping[['seat_id', 'section_name', 'row_number', 'seat_number']],
    how='left',
    left_on=['section_name', 'Row', 'Number'],
    right_on=['section_name', 'row_number', 'seat_number']
)

all_tickets['Seat ID'] = all_tickets['seat_id']
all_tickets.drop(columns=['seat_id', 'section_name', 'row_number', 'seat_number'], inplace=True)

# Load SF data
all_tickets_ids = all_tickets[['Owner Identifier', 'Account ID']].copy()
merged_ids = all_tickets_ids.merge(
    sf[['id__c', 'AccountId']].astype(str).apply(lambda x: x.str.strip()),
    how='left',
    left_on='Owner Identifier',
    right_on='id__c'
)
merged_ids['Account ID'] = merged_ids['Account ID'].fillna(merged_ids['AccountId'])
merged_ids.drop(columns=['id__c', 'AccountId'], inplace=True)
all_tickets['Account ID'] = merged_ids['Account ID']


all_tickets['Transaction'] = all_tickets['Transaction'].astype(int)

all_tickets

,Product ID,Season ID,Asset Name,Roboticket ID,Price,Price Name,Price Area Name,Base Price,From,To,...,Account ID,Contact,Product,Transaction,Return Asset,Status,Is Ticket,Is Season Ticket,Created By,Last Modified By
0,1331,24,שלב הבתים משחק 1: עירוני נס ציונה,3039242.0,-10.0,מחיר מנויים,F,0.0,None,None,...,NaN,None,None,3514971,None,Canceled,True,False,None,None
1,1331,24,שלב הבתים משחק 1: עירוני נס ציונה,3039245.0,-10.0,מחיר מנויים,F,0.0,None,None,...,NaN,None,None,3514971,None,Canceled,True,False,None,None
2,1331,24,שלב הבתים משחק 1: עירוני נס ציונה,3039244.0,-10.0,מחיר מנויים,F,0.0,None,None,...,NaN,None,None,3514971,None,Canceled,True,False,None,None
3,1331,24,שלב הבתים משחק 1: עירוני נס ציונה,3039243.0,-10.0,מחיר מנויים,F,0.0,None,None,...,NaN,None,None,3514971,None,Canceled,True,False,None,None
4,1331,24,שלב הבתים משחק 1: עירוני נס ציונה,3041925.0,-40.0,רגיל,A,0.0,None,None,...,NaN,None,None,3514741,None,Canceled,True,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4143,4205,24,חניה פנימית מכבי תל אביב 12.6,3428474.0,0.0,Complementary Parking,Parking,0.0,None,None,...,001Hu00003AEzoLIAT,None,None,3738960,None,Canceled,True,False,None,None
4144,4237,24,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,3439035.0,0.0,Complementary,יציע אורחים,0.0,None,None,...,NaN,None,None,3744815,None,Canceled,True,False,None,None
4145,4237,24,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,3439036.0,0.0,Complementary,יציע אורחים,0.0,None,None,...,NaN,None,None,3744814,None,Canceled,True,False,None,None
4146,4237,24,גמר ליגת ווינר משחק 3: מכבי תל אביב 🚗,3439038.0,0.0,Complementary,יציע אורחים,0.0,None,None,...,001Hu00003AEzoLIAT,None,None,3744813,None,Canceled,True,False,None,None


In [10]:
all_tickets.iloc[0]

Product ID                                           1331
Season ID                                              24
Asset Name              שלב הבתים משחק 1: עירוני נס ציונה
Roboticket ID                                   3039242.0
Price                                               -10.0
Price Name                                    מחיר מנויים
Price Area Name                                         F
Base Price                                            0.0
From                                                 None
To                                                   None
Stadium ID                                           None
Sector                                      אולם 1 עמידה 
Row                                                     C
Number                                                 23
Product Type                                        Match
Operation Name                                     Return
Product Name            שלב הבתים משחק 1: עירוני נס ציונה
Product Date  

In [4]:
installments = pd.read_csv('HapoelInstallmentsReportCsv_2025-08-13.csv')

installments = installments.rename(columns={'InstallmentTransactionId':'Transaction',
                                            'Installment Ticket Id': 'Return ID'})
installments

,Return ID,Transaction,InstallmentDate,InstallmentValueDate,InstallmentProducts,InstallmentPaymentPrice,InstallmentProductPrice,InstallmentPaymentExtRef,InstallmentProductExtRef,Installments
0,NaN,3596914,01/01/2025,01/01/2025,Other Payment,0.0,NaN,Other,NaN,NaN
1,3163501.0,3596914,01/01/2025,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN
2,3163500.0,3596914,01/01/2025,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN
3,NaN,3596915,01/01/2025,01/01/2025,Other Payment,0.0,NaN,Other,NaN,NaN
4,3163497.0,3596915,01/01/2025,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN
...,...,...,...,...,...,...,...,...,...,...
161588,3163489.0,3596912,31/12/2024,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN
161589,3163490.0,3596912,31/12/2024,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN
161590,3163491.0,3596912,31/12/2024,NaN,🏠 מחזור 13 (רדיוס בחולון): מכבי עירוני רמת גן,NaN,0.0,NaN,4114.0,NaN
161591,NaN,3596913,31/12/2024,31/12/2024,Other Payment,0.0,NaN,Other,NaN,NaN


In [5]:
# --- align + standardize ---
tickets  = all_tickets.copy()
installs = installments.copy()

tickets['Transaction']  = pd.to_numeric(tickets['Transaction'],  errors='coerce').astype('Int64')
tickets['Roboticket ID'] = pd.to_numeric(tickets['Roboticket ID'], errors='coerce').astype('Int64')
installs['Transaction'] = pd.to_numeric(installs['Transaction'], errors='coerce').astype('Int64')

# normalize ReturnID column name on installments
ret_cols = ['Return ID', 'Installment Ticket Id', 'Installment Ticket ID', 'ReturnID']
ret_col  = next(c for c in ret_cols if c in installs.columns)
installs = installs.rename(columns={ret_col: 'ReturnID'})

# --- keep only installments for transactions that exist in tickets ---
tx_df     = tickets[['Transaction']].drop_duplicates()
inst_kept = installs.merge(tx_df, on='Transaction', how='inner')

# --- assign within-transaction index on both sides and pair ---
tickets_ord = (
    tickets.sort_values(['Transaction','Roboticket ID'])
           .drop_duplicates(['Transaction','Roboticket ID'])
           .assign(k=lambda d: d.groupby('Transaction').cumcount())
)

inst_ord = (
    inst_kept.sort_values(['Transaction','ReturnID','InstallmentDate'])
             .assign(k=lambda d: d.groupby('Transaction').cumcount())
             [['Transaction','k','ReturnID']]   # add more cols here if you want them carried over
)

paired = tickets_ord.merge(inst_ord, on=['Transaction','k'], how='left', validate='one_to_one') \
                    .drop(columns=['k'])

paired['ReturnID'] = paired['ReturnID'].astype('Int64')

### Validation:

In [6]:
dups_global = (paired.dropna(subset=['ReturnID'])
                     .loc[lambda d: d.duplicated('ReturnID', keep=False)]
                     .sort_values('ReturnID'))
print("Global duplicates in paired:", dups_global['ReturnID'].nunique())


Global duplicates in paired: 0


In [7]:
paired

,Product ID,Season ID,Asset Name,Roboticket ID,Price,Price Name,Price Area Name,Base Price,From,To,...,Contact,Product,Transaction,Return Asset,Status,Is Ticket,Is Season Ticket,Created By,Last Modified By,ReturnID
0,1331,24,שלב הבתים משחק 1: עירוני נס ציונה,3031551,0.0,Complementary,F,0.0,None,None,...,None,None,3504213,None,Canceled,True,False,None,None,180814
1,1331,24,שלב הבתים משחק 1: עירוני נס ציונה,3033977,-40.0,רגיל,B,0.0,None,None,...,None,None,3507843,None,Canceled,True,False,None,None,181474
2,1331,24,שלב הבתים משחק 1: עירוני נס ציונה,3033034,-10.0,מחיר מנויים,A,0.0,None,None,...,None,None,3510611,None,Canceled,True,False,None,None,181738
3,1331,24,שלב הבתים משחק 1: עירוני נס ציונה,3033035,-10.0,מחיר מנויים,A,0.0,None,None,...,None,None,3510611,None,Canceled,True,False,None,None,181739
4,1331,24,שלב הבתים משחק 1: עירוני נס ציונה,3032119,-40.0,רגיל,A,0.0,None,None,...,None,None,3510842,None,Canceled,True,False,None,None,182101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4143,4172,24,גמר ליגת ווינר משחק 2: מכבי תל אביב 🏠,3435327,0.0,הזמנה,תא צפיה,0.0,None,None,...,None,None,3746676,None,Canceled,True,False,None,None,236888
4144,4172,24,גמר ליגת ווינר משחק 2: מכבי תל אביב 🏠,3435833,0.0,הזמנה,תא צפיה,0.0,None,None,...,None,None,3746678,None,Canceled,True,False,None,None,236889
4145,4172,24,גמר ליגת ווינר משחק 2: מכבי תל אביב 🏠,3435413,0.0,הזמנה,תא צפיה,0.0,None,None,...,None,None,3746680,None,Canceled,True,False,None,None,236890
4146,4172,24,גמר ליגת ווינר משחק 2: מכבי תל אביב 🏠,3428446,0.0,הזמנה VIP,Gold,0.0,None,None,...,None,None,3746682,None,Canceled,True,False,None,None,236891


In [8]:
paired.iloc[0]

Product ID                                           1331
Season ID                                              24
Asset Name              שלב הבתים משחק 1: עירוני נס ציונה
Roboticket ID                                     3031551
Price                                                 0.0
Price Name                                  Complementary
Price Area Name                                         F
Base Price                                            0.0
From                                                 None
To                                                   None
Stadium ID                                           None
Sector                                      אולם 1 עמידה 
Row                                                    15
Number                                                 15
Product Type                                        Match
Operation Name                                     Return
Product Name            שלב הבתים משחק 1: עירוני נס ציונה
Product Date  

In [9]:
paired.to_excel('tickets2425_with_returnID.xlsx')